##### Final Project on Adverse Drug Events for Asian American Demographic. It is important to note that the enriched dataset being predicted on, is built off inferred features from the FAERS dataset and NHANES dataset. Thus this prediction is more indicative of the dataset's internal logic, rather than confirming any external validity in the field of ADEs.

In [ ]:
#Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
